In [1]:
# This script was created to convert .csv files of wind data from https://portal.aoos.org/#map (Alaska Ocean Observing System)
# It will take the raw data and return a new file that has Sustained Wind Speed, Gust Factor, Year, Month, and Day.
# The Gust Factor is calculated using the sustained wind speed and gust wind speed values.

# Coded by: Brennan Dettmann
# Date: 6/5/2020

In [2]:
# imports all of the necessary modules

import csv
import os

In [3]:
# properly converts direction from degrees to cardinal direction

def directionconverter(drxn):
    drxn = int(float(drxn))    
    if drxn >= 337.5 or drxn < 22.5: return "N"
    elif drxn < 67.5: return "NE"
    elif drxn < 112.5: return "E"
    elif drxn < 157.5: return "SE"
    elif drxn < 202.5: return "S"
    elif drxn < 247.5: return "SW"
    elif drxn < 292.5: return "W"
    elif drxn < 337.5: return "NW"

In [4]:
# here is the meat of the script, I'll attempt to explain each subsection's purpose

# prompts the user to input a file name and defines the station and units (will be used when naming new files later)
filename = input('Input Filename: ') # make sure to put in format {station}_{units} ex. PAGN_kts
station = filename.split("_")[0]
units = filename.split("_")[1]

# uses the csv module to open the desired file
pulledFile = open(f'{filename}.csv')
reader = csv.reader(pulledFile, delimiter = ",")

# uses csv module to open a file where all of the data will go into
with open(f'/home/bdettman/Juneau_Internship/Wind-Gust-Climatology/sorteddata/{station}_All.csv', 'a') as allDir:
    allDirwriter = csv.writer(allDir, delimiter =',', quotechar='"', quoting = csv.QUOTE_ALL)
    allDirwriter.writerow(["Sustained Wind Speed (kts)", "Gust Factor", "Year", "Month", "Day"])
    
    for row in reader:     #this loop works to only read rows that have usuable data and pertain to the new files
        if row[2] != "0":
            if row[5] != "0":
                if row[2] != "kts":
                    if row[5] != "kts":
                        if row[2] != "wind_speed_kts":
                            if row[5] != "wind_speed_kts":
                                if row[2] != "#VALUE!":
                                    if row[5] != "#VALUE!":
                                        if row[3] != "NaN":
                                            if units == "mph":
                                                windSpeed = float(row[2])*.87
                                            if units == "knots" or units == "kts":
                                                windSpeed = float(row[2])
                                            windGust = float(row[5])
                                            gustMultiplier = windGust/windSpeed
                                            windDir = directionconverter(row[3])
                                            year = row[0][0:4]
                                            month = row[0][5:7]
                                            day = row[0][8:10]
                                            allDirwriter.writerow([windSpeed, gustMultiplier, year, month, day])     #writes all the data into the "all" file
                                            with open(f'/home/bdettman/Juneau_Internship/Wind-Gust-Climatology/sorteddata/{station}_{windDir}.csv', 'a') as oneDir:
                                                oneDirwriter = csv.writer(oneDir, delimiter =',', quotechar='"', quoting = csv.QUOTE_ALL)
                                                if os.path.getsize(f'/home/bdettman/Juneau_Internship/Wind-Gust-Climatology/sorteddata/{station}_{windDir}.csv') == 0:
                                                    oneDirwriter.writerow(["Sustained Wind Speed (kts)", "Gust Factor", "Year", "Month", "Day"])     #if the file does not exist yet, then it will write these variables at the top
                                                oneDirwriter.writerow([windSpeed, gustMultiplier, year, month, day])     #writes the data into each file based on the wind direction the row has

Input Filename:  MRNA2_kts


In [5]:
# closes the writer and reader modules

allDir.close()
allDir.close()
pulledFile.close()